# DEMO: Data Exploration and Feature Engineering

## Imports

In [404]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from snowflake.ml.modeling.preprocessing import *
from snowflake.ml.modeling.impute import *
import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Create Snowpark Session

In [405]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [406]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "MUSHROOMS_DB"."PUBLIC"
Current Warehouse: "MUSHROOM_M_WH"


## Snowpark DataFrames vs. Pandas DataFrames

In [407]:
# Creating a Pandas DataFrame
pandas_df = pd.read_csv('data/mushrooms.csv')
print(type(pandas_df))

<class 'pandas.core.frame.DataFrame'>


In [408]:
# Creating a Snowpark DataFrame
snowpark_df = session.table("MUSHROOM_DB.PUBLIC.MUSHROOMS") # add full path to table db.shema.table
print(type(snowpark_df))

<class 'snowflake.snowpark.table.Table'>


In [409]:
# Compare size
print('Size in MB of Pandas DataFrame in Memory:\n', np.round(sys.getsizeof(pandas_df) / (1024.0**2), 2))
print('Size in MB of Snowpark DataFrame in Memory:\n', np.round(sys.getsizeof(snowpark_df) / (1024.0**2), 2))

Size in MB of Pandas DataFrame in Memory:
 10.34
Size in MB of Snowpark DataFrame in Memory:
 0.0


The only thing stored in a Snowpark DataFrame is the SQL needed to return data

In [410]:
snowpark_df.queries

{'queries': ['SELECT  *  FROM (MUSHROOM_DB.PUBLIC.MUSHROOMS)'],
 'post_actions': []}

A Snowpark DataFrame can be converted to a Pandas DataFrame. This will pull the data from Snowflake into the Python enviroment memory.

In [411]:
pandas_df2= snowpark_df.to_pandas()

Both our Pandas DataFrames has the same shape

In [412]:
pandas_df.shape, pandas_df2.shape

((8124, 23), (8124, 23))

Showing a Snowpark DataFrame

In [413]:
pandas_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [414]:
# check for null vlaues
pandas_df2.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [415]:
#show all cols
pandas_df2.columns

Index(['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat'],
      dtype='object')

In [416]:
# snowpark_df.show() <- also possible
snowpark_df.limit(5).show() # <- collects first 5 rows and displays as pandas-dataframe

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"class"  |"cap-shape"  |"cap-surface"  |"cap-color"  |"bruises"  |"odor"  |"gill-attachment"  |"gill-spacing"  |"gill-size"  |"gill-color"  |"stalk-shape"  |"stalk-root"  |"stalk-surface-above-ring"  |"stalk-surface-below-ring"  |"stalk-color-above-ring"  |"stalk-color-below-ring"  |"veil-type"  |"veil-color"  |"ring-number"  |"ring-type"  |"spore-print-color"  |"population"  |"habitat"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Simple Transformations
Select specific columns

In [417]:
snowpark_df.select('"class"').show()


-----------
|"class"  |
-----------
|p        |
|e        |
|e        |
|p        |
|e        |
|e        |
|e        |
|e        |
|p        |
|e        |
-----------



To aggregate data the **group_by** method are used in combination with the **agg** method.

In [418]:
snowpark_df.group_by(['"class"']).count().show()


---------------------
|"class"  |"COUNT"  |
---------------------
|p        |3916     |
|e        |4208     |
---------------------



## Simple Data Analysis
In this section we will use API Snowpark to do some basic analysis of our data.  
Start by creating a new Snowpark DataFrame

The **count** method on a DataFrame will return the number of rows

In [419]:
# Number of rows in dataset
snowpark_df.count()

8124

If we want to filter out duplicated rows, keeping only one, we can use the **drop_duplicates** method.

In [420]:
# Lets drop duplicates based on ID
snowpark_df.drop_duplicates().count()


8124

Using the **describe** method will return some basic statistics for all numeric and string columns.

In [421]:
# Calculating various statistics per column
snowpark_df.describe().show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"class"  |"cap-shape"  |"cap-surface"  |"cap-color"  |"bruises"  |"odor"  |"gill-attachment"  |"gill-spacing"  |"gill-size"  |"gill-color"  |"stalk-shape"  |"stalk-root"  |"stalk-surface-above-ring"  |"stalk-surface-below-ring"  |"stalk-color-above-ring"  |"stalk-color-below-ring"  |"veil-type"  |"veil-color"  |"ring-number"  |"ring-type"  |"spore-print-color"  |"population"  |"habitat"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [422]:
#we see that stalk root has a question mark lets explore that
snowpark_df.group_by(['"stalk-root"']).count().show()


--------------------------
|"stalk-root"  |"COUNT"  |
--------------------------
|e             |1120     |
|c             |556      |
|b             |3776     |
|r             |192      |
|?             |2480     |
--------------------------



If we do not want to use specific columns we can use **drop** to remove those from a Snowpark DataFrame.  
**Note:** This is not removing them from the underlying table.

In [423]:
# let us just drop that root-stalk col
snowpark_df=snowpark_df.drop(['"stalk-root"'])

In [424]:
# we see that veil-type only has one value
snowpark_df.group_by(['"veil-type"']).count().show()

-------------------------
|"veil-type"  |"COUNT"  |
-------------------------
|p            |8124     |
-------------------------



In [425]:
#let u s drop that one also
snowpark_df=snowpark_df.drop(['"veil-type"'])

In [426]:
# Calculating various statistics per column
snowpark_df.describe().show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"class"  |"cap-shape"  |"cap-surface"  |"cap-color"  |"bruises"  |"odor"  |"gill-attachment"  |"gill-spacing"  |"gill-size"  |"gill-color"  |"stalk-shape"  |"stalk-surface-above-ring"  |"stalk-surface-below-ring"  |"stalk-color-above-ring"  |"stalk-color-below-ring"  |"veil-color"  |"ring-number"  |"ring-type"  |"spore-print-color"  |"population"  |"habitat"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [427]:
#snowpark_df.write.save_as_table(table_name='MUSCHROOM_CLEAN', mode='overwrite')
#session.table('MUSCHROOM_CLEAN').show()

In [428]:
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")

Current Database and schema: "MUSHROOMS_DB"."PUBLIC"
